In [18]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from pathlib import Path

import requests
from bs4 import BeautifulSoup

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
service = Service()
driver = webdriver.Chrome(service=service, options=options)

In [13]:
def scrape_euvsdisinfo_selenium(pages=10):
    base_url = "https://euvsdisinfo.eu/disinformation-cases/?sort=desc&numberposts=60"
    data = []

    for page in range(1, pages + 1):
        print(f"Sida {page}")
        driver.get(base_url + str(page))
        time.sleep(2.5)  # Vänta på att innehållet laddas

        items = driver.find_elements(By.CLASS_NAME, "disinfo-item")

        for item in items:
            try:
                disinfo = item.find_element(By.CLASS_NAME, "disinfo-text").text.strip()
                correction = item.find_element(By.CLASS_NAME, "disproof-text").text.strip()
                country = item.find_element(By.CLASS_NAME, "country").text.strip()
                date = item.find_element(By.CLASS_NAME, "date").text.strip()

                data.append({
                    "text": disinfo,
                    "correction": correction,
                    "country": country,
                    "date": date,
                    "label": 1
                })
            except Exception:
                continue

    return pd.DataFrame(data)

In [16]:
df = scrape_euvsdisinfo_selenium(pages=12)
driver.quit()
print(f'Hämtade {len(df)} desinfoposter.')

Sida 1
Sida 2
Sida 3
Sida 4
Sida 5
Sida 6
Sida 7
Sida 8
Sida 9
Sida 10
Sida 11
Sida 12
Hämtade 0 desinfoposter.


In [4]:
df.to_csv('euvsdisinfo.csv', index=False)
print('Sparat')

Sparat


In [11]:
def scrape_euvsdisinfo(pages=10):
    #base_url = "https://euvsdisinfo.eu/disinformation-cases/?text&date&per_page=60&page="
    base_url = 'https://euvsdisinfo.eu/disinformation-cases/?sort=desc&numberposts=60'
    data = []

    for page in range(1, pages + 1):
        print(f'Hämtar sida {page}')
        url = base_url + str(page)
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')

        items = soup.find_all('div', class_='disinfo-item')

        for item in items:
            try:
                disinfo = item.find('div', class_='disinfo-text').get_text(Strip=True)
                fact = item.find("div", class_="disproof-text").get_text(strip=True)
                country = item.find("span", class_="country").get_text(strip=True)
                date = item.find("span", class_="date").get_text(strip=True)

                data.append({
                    'text': disinfo,
                    'correction': fact,
                    'country': country,
                    'date': date,
                    'label': 1
                })
            except AttributeError:
                continue
    return pd.DataFrame(data)